In [2]:
import pandas as pd
import ast
import pandas as pd
import openai
import ast
import tqdm
import time
from multiprocessing import Pool
from openai.error import RateLimitError


pd.set_option('display.max_columns', None)

merged_df = pd.read_csv('merged_df.csv')

merged_df


,Institution UKPRN code,Institution name,Main panel_x,Unit of assessment number_x,Unit of assessment name,Multiple submission letter_x,Multiple submission name_x,Joint submission_x,REF impact case study identifier,Title,Is continued from 2014,Summary impact type,Countries,Formal partners,Funding programmes,Global research identifiers,Name of funders,Researcher ORCIDs,Grant funding,1. Summary of the impact,2. Underpinning research,3. References to the research,4. Details of the impact,5. Sources to corroborate the impact,COVID-19 Statement,Institution code (UKPRN),Institution sort order,Main panel_y,Unit of assessment number_y,Multiple submission letter_y,Multiple submission name_y,Joint submission_y,Profile,FTE of submitted staff,Total FTE of submitted staff for joint submission,% of eligible staff submitted,4*,3*,2*,1*,Unclassified,Weighted Avg,Variance,5 Class Labels,9 Class Labels,Number of doctoral degrees awarded in academic year 2013,Number of doctoral degrees awarded in academic year 2014,Number of doctoral degrees awarded in academic year 2015,Number of doctoral degrees awarded in academic year 2016,Number of doctoral degrees awarded in academic year 2017,Number of doctoral degrees awarded in academic year 2018,Number of doctoral degrees awarded in academic year 2019,full text processed,Details of Impact Processed,Details of Impact Processed Strings
0,10007144,University of East London,C,21.0,Sociology,NaN,NaN,NaN,1e6e075e-d5f5-421c-bb45-2708636b5190,"?Go Home?; Shaping the practice, policy and un...",0.0,Societal,[Italy];[Sweden];[Netherlands ];[Ireland];[Uni...,[University of Eastern Finland];[Middle East T...,[European Commission under the 7th Framework P...,NaN,[EU];[ESRC];[AHRC];[EPSRC];[Nuffield],[0000-0003-2682-9573];[0000-0002-2095-1407];[0...,[FP7-SSH-2011-1-290775: 233600];[ES/L008971/1:...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,NaN,10007144,42,C,21,NaN,NaN,NaN,Impact,23.60,NaN,85.0,16.7,33.3,50.0,0.0,0.0,2.667,0.556111,3,5,10.0,9.16,15.66,7.33,14.33,8.67,2.0,The Center for Migration Refugees and Belongin...,"[['since', 'cmrb', 'finding', 'use', 'substant...",since cmrb finding use substantially reshape k...
1,10007151,University of Lincoln,A,6.0,"Agriculture, Food and Veterinary Sciences",NaN,NaN,NaN,73be8e7c-907e-4e5b-8a8b-953b4d394c1b,Agricultural Robotics: Data Driven Technologie...,0.0,Technological,[England];[Norway];[USA],[SAGA Robotics];[Garford Farms Limited];[Berry...,[N/A],[grid.423443.6];[grid.418100.c],[Innovate UK];[BBSRC],[0000-0002-4297-4837];[0000-0003-2604-7555];[0...,[103339: 493842];[104587: 407867];[105151: 642...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,NaN,10007151,77,A,6,NaN,NaN,NaN,Impact,33.45,NaN,100.0,33.3,33.4,33.3,0.0,0.0,3.000,0.666000,3,6,2.0,1.00,6.00,3.00,5.00,7.00,1.0,The University of Lincoln has led the adoption...,"[['uols', 'innovative', 'agrifood', 'research'...",uols innovative agrifood research underpin ser...
2,10007788,University of Cambridge,A,4.0,"Psychology, Psychiatry and Neuroscience",NaN,NaN,NaN,d59a4fa1-5e19-4394-bf46-451d905f0fa6,Alleviating the societal and economic burden o...,0.0,Health,[England];[China];[USA];[Canada];[Belgium];[Fr...,NaN,NaN,[grid.14105.31];[grid.473755.7];[grid.52788.30],[Medical Research Council];[NIHR Central Commi...,[0000-0002-7304-2231];[0000-0002-0387-880X],[HQR00520: 472000];[NIHR: 150000];[16/137/105:...,**1. Summary of the impact** \(indicative maxi...,**2. Underpinning research \(indicative maximu...,**3. References to the research** \(indicative...,**4. Details of the impact** \(indicative maxi...,**5. Sources to corrobora

In [4]:
df = merged_df[["Institution name","Main panel_x","Unit of assessment number_x","Unit of assessment name", "Title", "Summary impact type","Countries","Formal partners","Funding programmes","Global research identifiers","Name of funders",'Researcher ORCIDs',"Grant funding","1. Summary of the impact","2. Underpinning research","3. References to the research","4. Details of the impact","5. Sources to corroborate the impact","Weighted Avg","Variance","5 Class Labels","9 Class Labels"]]

df

,Institution name,Main panel_x,Unit of assessment number_x,Unit of assessment name,Title,Summary impact type,Countries,Formal partners,Funding programmes,Global research identifiers,Name of funders,Researcher ORCIDs,Grant funding,1. Summary of the impact,2. Underpinning research,3. References to the research,4. Details of the impact,5. Sources to corroborate the impact,Weighted Avg,Variance,5 Class Labels,9 Class Labels
0,University of East London,C,21.0,Sociology,"?Go Home?; Shaping the practice, policy and un...",Societal,[Italy];[Sweden];[Netherlands ];[Ireland];[Uni...,[University of Eastern Finland];[Middle East T...,[European Commission under the 7th Framework P...,NaN,[EU];[ESRC];[AHRC];[EPSRC];[Nuffield],[0000-0003-2682-9573];[0000-0002-2095-1407];[0...,[FP7-SSH-2011-1-290775: 233600];[ES/L008971/1:...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,2.667,0.556111,3,5
1,University of Lincoln,A,6.0,"Agriculture, Food and Veterinary Sciences",Agricultural Robotics: Data Driven Technologie...,Technological,[England];[Norway];[USA],[SAGA Robotics];[Garford Farms Limited];[Berry...,[N/A],[grid.423443.6];[grid.418100.c],[Innovate UK];[BBSRC],[0000-0002-4297-4837];[0000-0003-2604-7555];[0...,[103339: 493842];[104587: 407867];[105151: 642...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,3.000,0.666000,3,6
2,University of Cambridge,A,4.0,"Psychology, Psychiatry and Neuroscience",Alleviating the societal and economic burden o...,Health,[England];[China];[USA];[Canada];[Belgium];[Fr...,NaN,NaN,[grid.14105.31];[grid.473755.7];[grid.52788.30],[Medical Research Council];[NIHR Central Commi...,[0000-0002-7304-2231];[0000-0002-0387-880X],[HQR00520: 472000];[NIHR: 150000];[16/137/105:...,**1. Summary of the impact** \(indicative maxi...,**2. Underpinning research \(indicative maximu...,**3. References to the research** \(indicative...,**4. Details of the impact** \(indicative maxi...,**5. Sources to corroborate the impact** \(ind...,3.667,0.444111,4,7
3,University of Cambridge,A,6.0,"Agriculture, Food and Veterinary Sciences",Changes in policy for the control of bovine tu...,Environmental,[England];[Wales];[Ethiopia],NaN,NaN,[grid.418100.c];[grid.434257.3];[grid.433527.4...,[BBSRC];[Economic and Social Research Council]...,[0000-0002-0258-3188];[0000-0002-2593-6353],[BB/L018977/1: 2900000];[SE3133 PO366620: 3235...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,3.500,0.450000,4,7
4,University of Derby,C,20.0,Social Work and Social Policy,Changing Police and Investigative Practice thr...,Legal,NaN,NaN,NaN,NaN,NaN,[0000-0001-6065-467X],NaN,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,2.333,1.554111,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6346,University of Strathclyde,B,11.0,Computer Science and Informatics,Understanding and removing barriers to the wid...,Health,[Scotland],NaN,NaN,NaN,[Innovate UK];[SFC],NaN,NaN,## 1. Summary of the impact \n\n Research int...,## 2. Underpinning research \n\n### Context ...,## 3. References to the research \(Strathclyd...,## 4. Details of the impact \n\n The Test of ...,## 5. Sources to corroborate the impact \n\n1...,3.333,0.222111,3,7
6347,The London School of Economi

In [6]:
import pandas as pd
import openai
import ast
import tqdm
import time
from multiprocessing import Pool
from openai.error import RateLimitError

openai.api_key = ""  

def process_text_with_gpt(text_chunk, retries=3, delay=1):
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant skilled in text formatting."},
                    {"role": "user", "content": f"Please reformat the following text by removing any formatting, headings, and citations. Maintain the original wording and structure without summarizing or altering the content. Ensure the word count remains roughly the same. Headings are usually short and introduce the next paragraph. Headings are usually short where they are found combine them with the next paragraph to form cohesive sentences. Here is the text:{text_chunk}"}
                ],
                max_tokens=2000  
            )
            return response.choices[0].message['content']
        except RateLimitError as e:
            print(f"Rate limit error encountered: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  #
        except Exception as e:
            print(f"An error occurred: {e}")
            break  

    return None  # If all retries fail

def process_text_by_paragraph_halves(text):
    paragraphs = text.split('\n\n')  #
    
    total_paragraphs = len(paragraphs)
    middle_index = total_paragraphs // 2
    
    # Split paragraphs into two halves
    first_half = paragraphs[:middle_index]
    second_half = paragraphs[middle_index:]
    
    first_half_text = '\n\n'.join(first_half)
    second_half_text = '\n\n'.join(second_half)
    
    processed_first_half = process_text_with_gpt(first_half_text.strip())
    processed_second_half = process_text_with_gpt(second_half_text.strip())
    
    # Combine the processed halves
    final_text = '\n\n'.join([processed_first_half, processed_second_half]) if processed_first_half and processed_second_half else None
    
    return final_text

def process_row(text):
    return process_text_by_paragraph_halves(text)


In [17]:
def main():    
    with Pool(4) as pool:  
        results = list(tqdm.tqdm(pool.imap(process_row, df['4. Details of the impact']), total=len(df), desc="Processing rows"))
    
    df['cleaned text'] = results

    df.to_csv('df_gpt4.csv', index=False)

In [18]:
main()


Processing rows:   0%|          | 1/6351 [00:13<24:02:22, 13.63s/it]Process ForkPoolWorker-3:



KeyboardInterrupt: 

12550